In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
schools_data = os.path.join('raw_data','schools_complete.csv')
students_data = os.path.join('raw_data', 'students_complete.csv')

schools_df = pd.read_csv(schools_data)
students_df = pd.read_csv(students_data)

In [3]:
schools_df=schools_df.rename(index=str, columns={"name": "school"})
schools_df.head()

School ID                 school      type  size   budget
0          0      Huang High School  District  2917  1910635
1          1   Figueroa High School  District  2949  1884411
2          2    Shelton High School   Charter  1761  1056600
3          3  Hernandez High School  District  4635  3022020
4          4    Griffin High School   Charter  1468   917500

In [4]:
students_df.head()

Student ID               name gender grade             school  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  
0             66          79  
1             94          61  
2             90          60  
3             67          58  
4             97          84

In [5]:
total_schools = len(schools_df['school'].unique())
total_students = len(students_df['Student ID'].unique())
total_budget = schools_df['budget'].sum()
avg_math_score = students_df['math_score'].mean()
avg_read_score = students_df['reading_score'].mean()

In [6]:
#defining pass score as 70+ for math and verbal
pct_pass_read = np.count_nonzero(students_df['reading_score']>=70)/total_students
pct_pass_math = np.count_nonzero(students_df['math_score']>=70)/total_students
overall_pass_rate = (pct_pass_math+pct_pass_read)/2

In [7]:
district_summary = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_read_score],
    "Pct Passing Reading":[pct_pass_read],
    "Pct Passing Math": [pct_pass_math],
    "Overall Passing Rate": [overall_pass_rate]
})
    
dist_sum = district_summary[["Total Schools",
    "Total Students",
    "Total Budget",
    "Average Math Score",
    "Average Reading Score",
    "Pct Passing Reading",
    "Pct Passing Math",
    "Overall Passing Rate"]]

In [8]:
dist_sum = dist_sum.style.format({
    "Total Budget": "${:,}",
    "Average Math Score": "{:.2f}",
    "Average Reading Score": "{:.2f}",
    "Pct Passing Reading":"{:.2%}",
    "Pct Passing Math": "{:.2%}",
    "Overall Passing Rate": "{:.2%}",
    "Total Students": "{:,}"
})

In [9]:
dist_sum

In [10]:
merged_df = pd.merge(students_df, schools_df, on="school", how="left")
school_group = merged_df.groupby(['school'])

In [11]:
## SCHOOL SUMMARY

group_per_school = merged_df.set_index('school').groupby(['school'])

types_per_school = schools_df.set_index('school')['type']

students_per_school = group_per_school['Student ID'].count()

budget_per_school = schools_df.set_index('school')['budget']

budget_per_student = schools_df.set_index('school')['budget']/schools_df.set_index('school')['size']

# Averages
avg_math = group_per_school['math_score'].mean()
avg_read = group_per_school['reading_score'].mean()

# Passing Scores
pass_math = merged_df[merged_df['math_score'] >= 70].groupby('school')['Student ID'].count()/students_per_school 
pass_read = merged_df[merged_df['reading_score'] >= 70].groupby('school')['Student ID'].count()/students_per_school 
overall = merged_df[(merged_df['reading_score'] >= 70) & (merged_df['math_score'] >= 70)].groupby('school')['Student ID'].count()/students_per_school 

school_summary = pd.DataFrame({
    "School Type": types_per_school,
    "Total Students": students_per_school,
    "Per Student Budget": budget_per_student,
    "Total School Budget": budget_per_school,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
})

school_summary = school_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          'Overall Passing Rate']]

#formatting
school_summary = school_summary.style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:.0f}",
                          'Average Math Score': "{:.1f}", 
                          'Average Reading Score': "{:.1f}", 
                          "% Passing Math": "{:.1%}", 
                          "% Passing Reading": "{:.1%}", 
                          "Overall Passing Rate": "{:.1%}"})

In [12]:
school_summary